In [1]:
import os
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import altair as alt
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import (
 cross_val_score,
 cross_validate,
 train_test_split,
 GridSearchCV,
 RandomizedSearchCV
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from xgboost import XGBRegressor
#import shap
# Save a vega-lite spec and a PNG blob for each plot in the notebook
#alt.renderers.enable('mimetype')

### Transform data 

In [2]:
X_train = pd.read_csv("../data/raw/X_train.csv", parse_dates=['year'])
X_train['year'] = X_train['year'].dt.year
y_train = pd.read_csv("../data/raw/y_train.csv")
train_df = X_train.join(y_train.set_index('carID'), on = "carID")

In [3]:
train_df.head()

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2,31995
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0,7700
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9,58990
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0,12999
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0,16990


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         4960 non-null   int64  
 1   brand         4960 non-null   object 
 2   model         4960 non-null   object 
 3   year          4960 non-null   int64  
 4   transmission  4960 non-null   object 
 5   mileage       4960 non-null   int64  
 6   fuelType      4960 non-null   object 
 7   tax           4960 non-null   float64
 8   mpg           4960 non-null   float64
 9   engineSize    4960 non-null   float64
 10  price         4960 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 426.4+ KB


In [5]:
drop_features = ["carID"]
categorical_features = ["brand", "model", "transmission", "fuelType"]
ordinal_features = ["year"]
target = "price"
numeric_features = list(
    set(train_df.columns)
    - set(drop_features)
    - set(categorical_features)
    - set(ordinal_features)
    - set([target])
)
assert train_df.columns.shape[0] == len(
    drop_features
    + categorical_features
    + numeric_features
    + ordinal_features
    + [target]
)

In [6]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", dtype="int"), categorical_features),
    (OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan), ordinal_features),
    ("drop", drop_features))

In [7]:
X_train, y_train = train_df.drop(columns=[target]), train_df[target]

In [8]:
scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2"}

In [9]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['tax', 'mpg', 'mileage', 'engineSize']),
                                ('onehotencoder',
                                 OneHotEncoder(dtype='int',
                                               handle_unknown='ignore'),
                                 ['brand', 'model', 'transmission',
                                  'fuelType']),
                                ('ordinalencoder',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=nan),
                                 ['year']),
                                ('drop', 'drop', ['carID'])])

### Models

#### baseline

In [10]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [11]:
results ={}

In [12]:
dummy = DummyRegressor()
results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)

In [13]:
pd.DataFrame(results)

,Dummy
fit_time,0.001 (+/- 0.001)
score_time,0.000 (+/- 0.000)
test_neg RMSE,-16394.964 (+/- 837.605)
train_neg RMSE,-16403.194 (+/- 211.166)
test_r2,-0.003 (+/- 0.002)
train_r2,0.000 (+/- 0.000)


In [14]:
pipe_lr = make_pipeline(
    preprocessor, Ridge(random_state=123)
)

pipe_ls = make_pipeline(
    preprocessor, Lasso(random_state=123)
)

pipe_rf = make_pipeline(
    preprocessor, RandomForestRegressor(random_state=123)
)

pipe_xgboost = make_pipeline(
    preprocessor, XGBRegressor(random_state=123)
)

In [15]:
models = {
    "Ridge": pipe_lr,
    "Lasso": pipe_ls,
    "Random Forest": pipe_rf,
    "XGBoost": pipe_xgboost
}

In [16]:
for (name, model) in models.items():
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
    )

In [17]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random Forest,XGBoost
fit_time,0.001 (+/- 0.001),0.023 (+/- 0.003),0.324 (+/- 0.127),4.075 (+/- 0.102),0.519 (+/- 0.076)
score_time,0.000 (+/- 0.000),0.004 (+/- 0.000),0.004 (+/- 0.000),0.024 (+/- 0.002),0.008 (+/- 0.001)
test_neg RMSE,-16394.964 (+/- 837.605),-5675.981 (+/- 337.106),-5658.618 (+/- 331.747),-3364.668 (+/- 157.326),-3262.875 (+/- 172.058)
train_neg RMSE,-16403.194 (+/- 211.166),-5530.901 (+/- 79.265),-5509.347 (+/- 80.505),-1361.142 (+/- 35.464),-1581.720 (+/- 57.040)
test_r2,-0.003 (+/- 0.002),0.880 (+/- 0.005),0.881 (+/- 0.006),0.958 (+/- 0.003),0.960 (+/- 0.003)
train_r2,0.000 (+/- 0.000),0.886 (+/- 0.001),0.887 (+/- 0.001),0.993 (+/- 0.000),0.991 (+/- 0.001)


In [18]:
#print(pipe_xgboost.get_params().keys())

In [19]:
params = { 'xgbregressor__max_depth': np.arange(1,20,1),
           'xgbregressor__learning_rate': np.linspace(0.01,1, 20),
          'xgbregressor__colsample_bytree': [0.3, 0.7]}

In [20]:
best_model = RandomizedSearchCV(
    pipe_xgboost, params, cv=3,  scoring="neg_root_mean_squared_error", return_train_score=True, random_state=123
)

In [21]:
best_model.fit(X_train, y_train) 
best_model

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['tax',
                                                                                'mpg',
                                                                                'mileage',
                                                                                'engineSize']),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(dtype='int',
                                                                                             handle_unknown='ignore'),
                                                                               ['brand',
                                                                                'model',
                                                                                'transmission',
                                                                                'fuelType']),
                                                                              ('ordinalencoder',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_...
                                        'xgbregressor__learning_rate': array([0.01      , 0.06210526, 0.11421053, 0.16631579, 0.21842105,
       0.27052632, 0.32263158, 0.37473684, 0.42684211, 0.47894737,
       0.53105263, 0.58315789, 0.63526316, 0.68736842, 0.73947368,
       0.79157895, 0.84368421, 0.89578947, 0.94789474, 1.        ]),
                                        'xgbregressor__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
                   random_state=123, return_train_score=True,
                   scoring='neg_root_mean_squared_error')

In [22]:
best_model.best_params_

{'xgbregressor__max_depth': 18,
 'xgbregressor__learning_rate': 0.11421052631578947,
 'xgbregressor__colsample_bytree': 0.3}

In [23]:
best_model.best_score_

-3256.977836068077

In [24]:
def final_model(X_train, y_train):
    params = { 'xgbregressor__max_depth': np.arange(1,20,1),
           'xgbregressor__learning_rate': np.linspace(0.01,1, 20),
          'xgbregressor__colsample_bytree': [0.3, 0.7]}
    
    best_model = RandomizedSearchCV(
        pipe_xgboost, 
        params, 
        cv=3,
        scoring="neg_root_mean_squared_error",
        return_train_score=True, 
        random_state=123)
    
    final_model = best_model.fit(X_train, y_train) 
    
    return final_model

In [26]:
preprocessor.fit(X_train, y_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['tax', 'mpg', 'mileage', 'engineSize']),
                                ('onehotencoder',
                                 OneHotEncoder(dtype='int',
                                               handle_unknown='ignore'),
                                 ['brand', 'model', 'transmission',
                                  'fuelType']),
                                ('ordinalencoder',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=nan),
                                 ['year']),
                                ('drop', 'drop', ['carID'])])

In [27]:
feature_names = (
    numeric_features
    + list(
        pipe_lr.named_steps["columntransformer"]
        .named_transformers_["onehotencoder"]
        .get_feature_names_out()
    )
    + ordinal_features)

In [28]:
X_train_encode = pd.DataFrame(
    data=preprocessor.transform(X_train).toarray(),
    columns=feature_names,
    index=X_train.index,
)
X_train_encode.head()

,tax,mpg,mileage,engineSize,brand_audi,brand_bmw,brand_ford,brand_hyundi,brand_merc,brand_skoda,...,transmission_Automatic,transmission_Manual,transmission_Other,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol,year
0,-0.088993,-0.295744,-0.848304,0.095814,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,22.0
1,-0.331725,0.272200,0.937516,-0.157579,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0
2,-0.088993,-0.595105,-0.810335,0.982689,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.0
3,-1.484700,0.202256,-0.185480,-0.157579,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,19.0
4,-0.088993,-0.091508,-0.875062,-1.424542,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,23.0


In [29]:
xgboost_explainer = shap.TreeExplainer(best_model.named_steps["xgbregressor"])
train_xgboost_shap_values = xgboost_explainer.shap_values(X_train_encode)

NameError: name 'shap' is not defined

In [ ]:
shap.summary_plot(train_xgboost_shap_values, X_train_encode, plot_type="bar")

In [ ]:
shap.summary_plot(train_xgboost_shap_values, X_train_encode)

In [ ]:
X_test = pd.read_csv("../data/raw/X_test.csv", parse_dates=['year'])
X_test['year'] = X_test['year'].dt.year
y_test = pd.read_csv("../data/raw/y_test.csv")

In [ ]:
X_test_encode = pd.DataFrame(
    data=preprocessor.transform(X_test).toarray(),
    columns=feature_names,
    index=X_test.index,
)

In [ ]:
test_catboost_shap_values = xgboost_explainer.shap_values(X_test_encode)

In [ ]:
X_test_encode = X_test_encode.round(2)

In [ ]:
shap.force_plot(
    xgboost_explainer.expected_value,
    test_catboost_shap_values[8],
    X_test_encode.loc[8, :],
    matplotlib=True,
)

In [ ]:
y_predict = pipe_xgboost.predict(X_test)

In [ ]:
y_predict

In [ ]:
r2_score(y_test['price'], y_predict)